In [8]:
import numpy as np
import glob, json, os, sys
from datetime import datetime, timedelta
import requests
import pandas as pd

## Set your user token
token = ''

### Set "vid" and time range "st", "en"

In [9]:
st = str(int(datetime(2024,12,14,0).timestamp()*1000))
en = str(int(datetime(2024,12,18,0).timestamp()*1000))
vid = '1207341245353951232'
vid = '1262204802662662144'

### Get FR Logs

In [10]:
url = f"https://apis.intangles.com/fr_logs/get?vehicle_id={vid}&from={st}&to={en}&token={token}&psize=1000"
payload = {}
headers = {}
frResponse = requests.request("GET", url, headers=headers, data=payload)

print(frResponse.status_code)
frLogs = frResponse.json().get('result',{}).get('logs',[])

200


### Get Driver Session Logs

In [11]:
vids = [vid]
url = "https://apis.intangles.com/driver_session/sessions?start_time={st}&end_time={en}&account_id=1205841794575630336&psize=500&vids={vids}"
# driver_ids=1307349345628061696
# status=closed&
# &end_time={en}
url = url.format(vids=','.join(vids), st=st, en=en) + '&token='+token

payload = {}
headers = {}

drvResponse = requests.request("GET", url, headers=headers, data=payload)

print(drvResponse.status_code)
drvSess = drvResponse.json().get('result',{})
len(drvSess)

200


27

### Convert timestamp to datetime and Get Logs as DataFrame

In [12]:
pd.set_option('display.max_rows',20)
tags = ['imei', 'match_type', 'matched_driver', 'matched_driver_type', 'vehicle_id', 'trigger_ts', '__lastcreatedtime']
frDF = pd.DataFrame.from_dict(frLogs)
frDF = frDF[tags]
frDF['time'] = [datetime.fromtimestamp(t/1000) for t in frDF['trigger_ts'].values]
frDF['frCreated'] = [datetime.fromtimestamp(t/1000) for t in frDF['__lastcreatedtime'].values]
tagsMod = tags = ['imei', 'match_type', 'matched_driver', 'matched_driver_type', 'vehicle_id', 'time', 'frCreated']
frDF = frDF[tagsMod]
frDF

,imei,match_type,matched_driver,matched_driver_type,vehicle_id,time,frCreated
0,C4123480014,smart_fr,1290195959740891136,unverified_drivers,1262204802662662144,2024-12-17 16:22:37,2024-12-17 16:22:42.264
1,C4123480014,smart_fr,1290195959740891136,unverified_drivers,1262204802662662144,2024-12-17 15:51:35,2024-12-17 15:51:39.386
2,C4123480014,smart_fr,1290195959740891136,unverified_drivers,1262204802662662144,2024-12-17 15:02:25,2024-12-17 15:44:59.219
3,C4123480014,smart_fr,1290195959740891136,unverified_drivers,1262204802662662144,2024-12-17 14:32:05,2024-12-17 15:42:43.043
4,C4123480014,smart_fr,1290195959740891136,unverified_drivers,1262204802662662144,2024-12-17 14:01:36,2024-12-17 15:40:20.601
...,...,...,...,...,...,...,...
55,C4123480014,smart_fr,1307349345628061696,unverified_drivers,1262204802662662144,2024-12-14 08:54:25,2024-12-14 08:54:30.077
56,C4123480014,smart_fr,1307349345628061696,unverified_drivers,1262204802662662144,2024-12-14 08:22:03,2024-12-14 08:22:18.596
57,C4123480014,smart_fr,1307349345628061696,unverified_drivers,1262204802662662144,2024-12-14 07:49:56,2024-12-14 07:50:01.453
58,C4123480014,smart_fr,1307349345628061696,unverified_drivers,1262204802662662144,2024-12-14 06:38:13,2024-12-14 06:38:39.386


In [13]:
pd.set_option('display.max_row',20)
df = pd.DataFrame.from_dict(drvSess)
drvCols = ['id', 'driver_id', 'driver_type', 'duration', 'start_time', 'end_time', 'status', 'vehicle_plate', 'vehicle_id', '__lastcreatedtime']
drvDF = df[drvCols]
drvDF['start'] = [datetime.fromtimestamp(t/1000) for t in drvDF['start_time'].values]
drvDF['end'] = [datetime.fromtimestamp(t/1000) for t in drvDF['end_time'].values]
drvDF['tCreate'] = [datetime.fromtimestamp(t/1000) for t in drvDF['__lastcreatedtime'].values]
drvColsMod = ['id', 'driver_id', 'driver_type', 'duration', 'start', 'end', 'status', 'vehicle_plate', 'vehicle_id', 'tCreate']
drvDF = drvDF[drvColsMod]
drvDF.iloc[-26:]

C:\Users\Vikram.Melapudi\AppData\Local\Temp\ipykernel_5760\2308163819.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drvDF['start'] = [datetime.fromtimestamp(t/1000) for t in drvDF['start_time'].values]
C:\Users\Vikram.Melapudi\AppData\Local\Temp\ipykernel_5760\2308163819.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drvDF['end'] = [datetime.fromtimestamp(t/1000) for t in drvDF['end_time'].values]
C:\Users\Vikram.Melapudi\AppData\Local\Temp\ipykernel_5760\2308163819.py:7: SettingWithCopyWarning: 

,id,driver_id,driver_type,duration,start,end,status,vehicle_plate,vehicle_id,tCreate
1,1370071432897232896,1290195959740891136,unverified_drivers,5375.455,2024-12-17 13:48:34.231,2024-12-17 15:18:09.686,closed,OD29M5907,1262204802662662144,2024-12-17 16:14:44.299
2,1370034503346552832,None,None,4644.141,2024-12-17 11:44:23.005,2024-12-17 13:01:47.146,closed,OD29M5907,1262204802662662144,2024-12-17 13:47:59.608
3,1369984650411245568,None,None,3300.440,2024-12-17 10:24:58.853,2024-12-17 11:19:59.293,closed,OD29M5907,1262204802662662144,2024-12-17 10:29:53.742
4,1369974526888116224,None,None,1952.881,2024-12-17 09:46:19.805,2024-12-17 10:18:52.686,closed,OD29M5907,1262204802662662144,2024-12-17 09:49:40.106
5,1369970913407139840,None,None,122.063,2024-12-17 09:32:05.242,2024-12-17 09:34:07.305,closed,OD29M5907,1262204802662662144,2024-12-17 09:35:18.585
...,...,...,...,...,...,...,...,...,...,...
22,1369552979052462080,1307349345628061696,unverified_drivers,4888.055,2024-12-14 13:54:33.714,2024-12-14 15:16:01.769,closed,OD29M5907,1262204802662662144,2024-12-16 05:54:35.270
23,1369542370214805504,1307349345628061696,unverified_drivers,4398.790,2024-12-14 11:45:22.009,2024-12-14 12:58:40.799,closed,OD29M5907,1262204802662662144,2024-12-16 05:12:25.926
24,1369532765422223360,1307349345628061696,unverified_drivers,4398.302,2024-12-14 10:00:01.631,2024-12-14 11:13:19.933,closed,OD29M5907,1262204802662662144,2024-12-16 04:34:15.965
25,1369519761729257472,1307349345628061696,unverified_drivers,4763.438,2024-12-14 07:40:47.690,2024-12-14 09:00:11.128,closed,OD29M5907,1262204802662662144,2024-12-16 03:42:35.643


### Merge by time to see in sequence

In [14]:
mDF = pd.merge(drvDF[['start','driver_id','end','duration','tCreate']], frDF[['time','matched_driver','frCreated']], how='outer', left_on='start', right_on='time', sort=True)

In [15]:
pd.set_option('display.max_row',500)
display(mDF)

,start,driver_id,end,duration,tCreate,time,matched_driver,frCreated
0,2024-12-14 05:59:27.328,1307349345628061696,2024-12-14 07:10:42.279,4274.951,2024-12-16 03:03:01.138,NaT,NaN,NaT
1,NaT,NaN,NaT,NaN,NaT,2024-12-14 06:06:32,1307349345628061696,2024-12-14 06:08:17.673
2,NaT,NaN,NaT,NaN,NaT,2024-12-14 06:38:13,1307349345628061696,2024-12-14 06:38:39.386
3,2024-12-14 07:40:47.690,1307349345628061696,2024-12-14 09:00:11.128,4763.438,2024-12-16 03:42:35.643,NaT,NaN,NaT
4,NaT,NaN,NaT,NaN,NaT,2024-12-14 07:49:56,1307349345628061696,2024-12-14 07:50:01.453
5,NaT,NaN,NaT,NaN,NaT,2024-12-14 08:22:03,1307349345628061696,2024-12-14 08:22:18.596
6,NaT,NaN,NaT,NaN,NaT,2024-12-14 08:54:25,1307349345628061696,2024-12-14 08:54:30.077
7,2024-12-14 10:00:01.631,1307349345628061696,2024-12-14 11:13:19.933,4398.302,2024-12-16 04:34:15.965,NaT,NaN,NaT
8,NaT,NaN,NaT,NaN,NaT,2024-12-14 10:07:39,1307349345628061696,2024-12-14 10:07:46.902
9,NaT,NaN,NaT,NaN,NaT,2024-12-14 10:40:24,1307349345628061696,2024-12-14 10:40:28.064


### Count missed FRLogs (between sessions)

In [16]:
drvDF = drvDF.sort_values('start')
frDF = frDF.sort_values('time')

missedFRs = []
pSess = drvDF.iloc[0]
for n in range(1,len(drvDF)):
    cSess = drvDF.iloc[n]
    st = pSess['end']
    en = cSess['start']
    # print(pSess['start'],'--',st,'         ',en,'--',cSess['end'])
    # check if there is any FR session between 2 driver session
    lFrDF = frDF[(frDF['time']>st)*(frDF['time']<en)]
    if len(lFrDF)>0:
        missedFRs.append(lFrDF)
    pSess = cSess
    
if frDF.iloc[-1]['time']>drvDF.iloc[-1]['end']:
    st = drvDF.iloc[-1]['end']
    lFrDF = frDF[frDF['time']>st]
    missedFRs.append(lFrDF)

nMissedFRs = sum([len(x) for x in missedFRs])

print('# missed FRs: ', nMissedFRs)

# missed FRs:  0
